In [1]:
import pandas as pd
import numpy as np
import re

In this notebook, we load and process The US Traffic Accident dataset for analysis and modeling. We clean the data and also extract some potentially useful information.

## 1. Dataset Overview

The US Traffic Accident dataset from [Kaggle](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents) provides comprehensive information on traffic accidents across the United States from 2016 to 2023. It contains 7,728,394 rows and 24 columns, each representing a different attribute of the accidents. Here's a brief overview of some of the key columns:

* `ID`: Unique identifier for each accident.
* `Source`: Source of the accident report (e.g., 911 call, news).
* `Severity`: Accident severity rating (on a scale from 1 to 4). 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).
* `Start_Time`: Start time of the accident.
* `End_Time`: Time when the impact of accident on traffic flow was dismissed.
* `Start_Lat/Start_Lng`: Latitude and longitude where the accident started.
* `End_Lat/End_Lng`: Latitude and longitude where the accident ended (many missing values).
* `Distance(mi)`: The length of the road extent affected by the accident.
* `Description`: Brief description of the accident.
* `Street`, `City`, `County`, `State`, `Zipcode`: Location details of the accident.
* `Country`: Country where the accident occurred (all should be the USA).
* `Timezone`: Timezone of the accident location.
* `Airport_Code`: Nearest airport to the accident location.
* `Weather_Timestamp`: Time when weather data was recorded.
* `Temperature(F)`, `Wind_Chill(F)`, `Humidity(%)`, `Pressure(in)`, `Visibility(mi)`, `Wind_Direction`, `Wind_Speed(mph)`, `Precipitation(in)`, `Weather_Condition`: Various weather-related attributes.
* `Amenity`, `Bump`, `Crossing`, `Give_Way`, `Junction`, `No_Exit`, `Railway`, `Roundabout`, `Station`, `Stop`, `Traffic_Calming`, `Traffic_Signal`, `Turning_Loop`: Boolean indicators for the presence of specific road features.
* `Sunrise_Sunset`, `Civil_Twilight`, `Nautical_Twilight`, `Astronomical_Twilight`: Time of day indicators related to the position of the sun.

## 2. ETL Steps

### 2.1. Data Extraction

In [2]:
data = pd.read_csv("data/US_Accidents_March23.csv")
data.shape

(7728394, 46)

In [3]:
init_nrows = data.shape[0]

### 2.2. Data Transformation

#### Drop Redundant and Irrelevant columns
Certain columns may not add much value to the predictive modeling process such as `End_Lat`, `Zipcode`, `Airport_code`, etc.

In [4]:
df_clean = data.drop(columns=[
    "Source", "End_Lat", "End_Lng", "Description", "Country", "Airport_Code", "Civil_Twilight", "Wind_Direction",
    "Nautical_Twilight", "Astronomical_Twilight", "Wind_Chill(F)", "Timezone", "Weather_Timestamp", "Zipcode"]).copy()

In [5]:
del(data)

#### Fix Data Types

In [6]:
df_clean["Start_Time"] = pd.to_datetime(df_clean["Start_Time"])
df_clean["End_Time"] = pd.to_datetime(df_clean["End_Time"])

#### Remove Duplicates
There are 140899 (0.18%) duplicated entries in the dataset.

In [7]:
df_clean.drop(columns=["ID"]).duplicated().sum()# / len(df_clean)

140899

In [8]:
df_clean = df_clean.drop(columns=["ID"]).drop_duplicates()

#### Handle Missing Values

In [10]:
missing_values = df_clean.isnull().sum()
missing_values = missing_values[missing_values > 0]

In [13]:
# Print percentage of missing values per column
100 * missing_values / df_clean.shape[0]

Street                0.139559
City                  0.003295
Temperature(F)        2.101590
Humidity(%)           2.233003
Pressure(in)          1.803599
Visibility(mi)        2.273899
Wind_Speed(mph)       7.445606
Precipitation(in)    28.850615
Weather_Condition     2.226453
Sunrise_Sunset        0.297555
dtype: float64

Some columns are missing a very small percentage of their data (less than 2%). We could safely remove those rows.

In [14]:
df_clean = df_clean[~df_clean["Street"].isnull()]
df_clean = df_clean[~df_clean["City"].isnull()]
df_clean = df_clean[~df_clean["Temperature(F)"].isnull()]
df_clean = df_clean[~df_clean["Visibility(mi)"].isnull()]
df_clean = df_clean[~df_clean["Pressure(in)"].isnull()]
df_clean = df_clean[~df_clean["Humidity(%)"].isnull()]
df_clean = df_clean[~df_clean["Weather_Condition"].isnull()]

Some rows in our dataset are missing day/time information in `Sunrise_Sunset`. We could use manually fill the missing values using the `Time` data. However, given that very few rows are missing this information (~0.3%), we could just drop them.

In [15]:
df_clean = df_clean[~df_clean["Sunrise_Sunset"].isnull()]

~28\% of the data is missing values for `Precipitation(in)`. This is a significant amount so we could use imputation to fill the missing values. For instance, missing precipiation could mean there were no rain. To validate this, we could use `Weather_Condition` after more processing.

About 7\% of the data is missing `Wind_Speed(mph)` and 2\% is missing `Wind_Direction`. Similarly to `Precipitatiom(in)`, we could use information from `Weather_Category` to fill the missing values after more processing.

#### Fix Inconsistencies

In [16]:
df_clean.describe()

,Severity,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.327531e+06,7.327531e+06,7.327531e+06,7.327531e+06,7.327531e+06,7.327531e+06,7.327531e+06,7.327531e+06,6.919538e+06,5.297867e+06
mean,2.213746e+00,3.617201e+01,-9.475451e+01,5.502983e-01,6.178422e+01,6.480563e+01,2.954520e+01,9.094921e+00,7.700647e+00,8.068156e-03
std,4.867966e-01,5.091290e+00,1.735555e+01,1.757743e+00,1.898521e+01,2.280620e+01,9.961187e-01,2.676606e+00,5.407158e+00,9.224145e-02
min,1.000000e+00,2.455480e+01,-1.246238e+02,0.000000e+00,-4.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.336993e+01,-1.172294e+02,0.000000e+00,4.900000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,3.578004e+01,-8.784453e+01,2.400000e-02,6.400000e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.008551e+01,-8.039143e+01,4.470000e-01,7.600000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.040000e+01,0.000000e+00
max,4.000000e+00,4.900220e+01,-6.711317e+01,4.417500e+02,2.030000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01


The values for some of the numerical variables are unrealistic, namely `Temperature(F)`, `Distance(mi)`, `Pressure(in)`, `Visibility(mi)`, and `Wind_Speed(mph)`.
For instance, the highest recorded temperature on Earth is around 134°F so a max of 203 is unrealistic. Similarly, the highest wind speeds recorded in hurricanes and tornadoes are well below 1087 mph.

We can discard some inomalies and outliers for the numerical variables based on the following real-world observations:
* Temperatures as low as -60 °F can be observed in extreme cold regions, while temperatures up to 130 °F can be seen in extreme hot regions.
* Normal atmospheric pressure at sea level ranges from about 27 to 31 inches of mercury.
* Visibility can drop to 0 in dense fog, and clear conditions might extend visibility to about 20 miles.
* Typical wind speeds can vary from calm (0 mph) to extreme conditions like hurricanes and tornadoes, which can reach up to 150 mph.

In [17]:
print(f'{100 * (df_clean["Temperature(F)"] > 130).sum() / df_clean.shape[0]:.5f}% of the data reports a temperature of over 130F')
print(f'{100 * ((df_clean["Pressure(in)"] < 25) | (df_clean["Pressure(in)"] > 31)).sum() / df_clean.shape[0]:.2f}% of the data reports an atmospheric pressure outside of normal (<27 or >31)')
print(f'{100 * (df_clean["Visibility(mi)"] > 20).sum() / df_clean.shape[0]:.5f}% of accident occurred with a visibility of 20 miles or more')
print(f'{100 * (df_clean["Wind_Speed(mph)"] > 150).sum() / df_clean.shape[0]:.5f}% of the data reports a wind speeds of over 150mph')

0.00053% of the data reports a temperature of over 130F
1.26% of the data reports an atmospheric pressure outside of normal (<27 or >31)
0.13096% of accident occurred with a visibility of 20 miles or more
0.00066% of the data reports a wind speeds of over 150mph


In [18]:
df_clean = df_clean[df_clean["Temperature(F)"] <= 130]
df_clean = df_clean[(df_clean["Pressure(in)"] >= 25) & (df_clean["Pressure(in)"] <= 31)]
df_clean = df_clean[df_clean["Visibility(mi)"] <= 20]
df_clean = df_clean[df_clean["Wind_Speed(mph)"] <= 150]

#### Feature Extraction and Engineering

There are 139 unique values of `Weather_Condition` in the dataset but some of them refer to the same condition due to typos and different wordings. We can group them into `Weather_Category` (ex. cloudy, clear condition, precipitation, etc.)

In [19]:
df_clean["Weather_Condition"].unique().shape

(139,)

In [20]:
def extract_weather_category(condition):
    regular_weather = "(cloud|overcast|rain|drizzle|thunderstorm|thunder|t-storm|tornado|snow|haze|fog|mist|smoke|sand|dust|hail|squalls|ice pellets|sleet|wintry mix)"
    special_weather = "(fair|clear|volcanic ash|fair windy|showers in the vicinity|n/a precipitation|fair)"
    match = re.search(special_weather, condition.lower())
    if match:
        return match.group(1)
    match = re.search(regular_weather, condition.lower())
    if match:
        return match.group(1)
    return "other"

In [21]:
weather_categories = {
    "cloud": "cloudy",
    "overcast": "cloudy",
    "fair": "clear condition",
    "clear": "clear condition",
    "rain": "precipitation",
    "drizzle": "precipitation",
    "showers in the vicinity": "precipitation",
    "t-storm": "thunderstorm",
    "thunder": "thunderstorm",
    "thunderstorm": "thunderstorm",
    "wintry mix": "precipitation",
    "n/a precipitation": "precipitation",
    "mist": "precipitation",
    "sleet": "precipitation",
    "ice pellets": "precipitation",
    "hail": "precipitation",
    "snow": "snowstorm",
    "fog": "visibility issue",
    "haze": "visibility issue",
    "smoke": "visibility issue",
    "dust": "visibility issue",
    "sand": "visibility issue",
    "volcanic ash": "visibility issue",
    "tornado": "extreme condition",
    "squalls": "extreme condition",
}

In [22]:
df_clean["Weather_Category"] = df_clean["Weather_Condition"].apply(extract_weather_category)
df_clean["Weather_Category"] = df_clean["Weather_Category"].replace(weather_categories)

In [23]:
df_clean["Weather_Category"].value_counts()

clear condition      3048710
cloudy               2881593
precipitation         510660
visibility issue      172144
snowstorm             138043
thunderstorm           70614
extreme condition         87
Name: Weather_Category, dtype: int64

`Street` names by themselves may have too many values to work with but could extract the road type (local road or highway) from the names.

In [24]:
def is_highway(street_name):
    # Check if the street name contains common highway prefixes or suffixes
    highway_prefixes = ['I-', 'US-', 'SR-', 'HWY', 'INTERSTATE', 'US HIGHWAY', 'STATE ROUTE']
    highway_suffixes = ['INTERSTATE', 'HIGHWAY', 'EXPRESSWAY', 'TURNPIKE', 'PARKWAY', 'ROUTE']
    for prefix in highway_prefixes:
        if street_name.upper().startswith(prefix):
            return True
    for suffix in highway_suffixes:
        if street_name.upper().endswith(suffix):
            return True
    return False

In [25]:
df_clean["Is_Highway"] = df_clean["Street"].apply(is_highway)

In [26]:
df_clean["Is_Highway"].value_counts()

False    4915957
True     1905894
Name: Is_Highway, dtype: int64

We can also compute the duration of the accident as the difference between `End_Time` and `Start_Time`.

In [27]:
df_clean["Duration(min)"] = (df_clean["End_Time"] - df_clean["Start_Time"]).dt.total_seconds() / 60

3948 (0.054\%) of reported accident in the dataset had an impact on traffic flow for more than 24h (which is unusual). Those are either anomalies in the dataset or extremely rare cases. We could therefore remove these rows.

In [28]:
df_clean = df_clean[df_clean["Duration(min)"] <= 24*60*60]

We could also extract time-related information from `Start_Time`.

In [29]:
df_clean["Date"] = pd.to_datetime(df_clean["Start_Time"].dt.date)
df_clean["Hour"] = df_clean["Start_Time"].dt.hour
df_clean["Day"] = df_clean["Start_Time"].dt.day
df_clean["Day_of_Week"] = df_clean["Start_Time"].dt.dayofweek + 1  # index for day of the week start at 0
df_clean["Month"] = df_clean["Start_Time"].dt.month
df_clean["Year"] = df_clean["Start_Time"].dt.year

`Sunrise_Sunset` can be used to determine if it was day of night.

In [30]:
df_clean = df_clean.rename(columns={"Sunrise_Sunset": "Is_Night"})
df_clean["Is_Night"] = df_clean["Is_Night"] == "Night"

In [31]:
df_clean["Is_Night"].value_counts()

False    4786134
True     2031869
Name: Is_Night, dtype: int64

#### Fill Missing Values By Imputation

We set data points with missing `Precipitation(in)` where the `Weather_Category` isn't a precipitation to 0.

In [32]:
df_clean.loc[(df_clean["Precipitation(in)"].isnull()) & (df_clean["Weather_Category"].str not in {"precipitation", "snowstorm", "thunderstorm"}), "Precipitation(in)"] = 0

We fill the missing values of `Wind_Speed` with the median for each corresponding `Weather_Category`.

In [33]:
for category in df_clean["Weather_Category"].unique():
    median = df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"].median()
    df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"] = df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"].fillna(median)

In [34]:
print(f"{100 * (init_nrows - df_clean.shape[0]) / init_nrows:.2f}% of the original dataset was dropped")

11.78% of the original dataset was dropped


In [35]:
df_clean.isnull().sum()

Severity             0
Start_Time           0
End_Time             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
Street               0
City                 0
County               0
State                0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Is_Night             0
Weather_Category     0
Is_Highway           0
Duration(min)        0
Date                 0
Hour                 0
Day                  0
Day_of_Week          0
Month                0
Year                 0
dtype: int64

### 2.4. Save Data

In [37]:
%time
df_clean.to_csv("data/US_Accidents_March23_Clean.csv", index=False)